In [1]:
import params
from google.cloud import datastore, storage, logging
import time
import pickle
import hashlib
import sys
import numpy as np
import portfolioGeneration

In [6]:
import importlib
importlib.reload(portfolioGeneration)

<module 'portfolioGeneration' from '/Users/patrickogrady/Documents/CODE/walkforwardTrader/portfolioGeneration.py'>

In [2]:
import portfolio
import dataAck
import warnings
import numpy as np
warnings.filterwarnings("ignore")
tickerToTrade = "EWZ"
allModels = portfolio.getModels(tickerToTrade, returnEntireObject=True)
for modelEntity in allModels:
    model = modelEntity["model"]
    print(model.describe())

((22, ('GDX', 32, None, None), 2, 'EWZ'), 22, 50)
((22, ('GDX', 32, None, None), 5, 'EWZ'), 22, 50)
((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 25)
((22, ('GDX', 32, None, None), 2, 'EWZ'), 22, 250)
((10, ('GDX', 32, None, None), 5, 'EWZ'), 10, 25)
((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100)
((10, ('GDX', 32, None, None), 2, 'EWZ'), 10, 100)
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 250)
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50)


In [3]:
uniqueModels = portfolioGeneration.getUniqueModels(allModels)
for model in uniqueModels:
    print(model.describe())

((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100)
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50)


# VIEW CORRELATION AND COVARIANCE

In [5]:
tickersRequired = []
tickersTraded = []
for mod in uniqueModels:
    
    print(mod.describe())
    if mod.inputSeries.targetTicker not in tickersRequired:
        tickersRequired.append(mod.inputSeries.targetTicker)
    if mod.inputSeries.series.ticker not in tickersRequired:
        tickersRequired.append(mod.inputSeries.series.ticker)
    if mod.inputSeries.targetTicker not in tickersTraded:
        tickersTraded.append(mod.inputSeries.targetTicker)
    

pulledData, validTickers = dataAck.downloadTickerData(tickersRequired)

joinedData = dataAck.joinDatasets([pulledData[ticker] for ticker in pulledData])
modelReturns, modelPredictions = portfolioGeneration.generateAllReturns(uniqueModels, joinedData)

((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100)
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50)
GDX
EWZ
((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100)


KeyboardInterrupt: 

In [18]:
modelReturns

,"((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100)","((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50)"
2009-03-02,0.003404,0.013616
2009-03-03,0.000000,0.035261
2009-03-04,-0.006623,-0.013245
2009-03-05,-0.001074,-0.000000
2009-03-06,-0.006638,-0.000000
2009-03-09,0.078182,0.000000
2009-03-10,-0.002361,-0.000000
2009-03-11,0.032116,0.016058
2009-03-12,-0.006223,-0.000000
2009-03-13,-0.007910,-0.000000


In [19]:
modelPredictions

,"((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100)","((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50)"
2009-03-02,0.25,1.00
2009-03-03,0.00,0.50
2009-03-04,0.25,0.50
2009-03-05,0.50,0.00
2009-03-06,0.50,0.00
2009-03-09,1.00,0.00
2009-03-10,1.00,0.00
2009-03-11,1.00,0.50
2009-03-12,1.00,0.00
2009-03-13,1.00,0.00


In [8]:
modelReturnsWithFactor = modelReturns.join(dataAck.getDailyFactorReturn(tickerToTrade, joinedData))

NameError: name 'modelReturns' is not defined

In [7]:
%matplotlib inline
portfolioGeneration.visualizeModels(modelReturnsWithFactor)

NameError: name 'modelReturnsWithFactor' is not defined

# STORE PAST MODEL PREDICTIONS

In [24]:
portfolioGeneration.storePastPredictions(uniqueModels, modelPredictions)

((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100)
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50)


# STORE MODELS AS PORTFOLIO

In [26]:
portfolioGeneration.storePortfolio(uniqueModels, description="Just trade underlying", benchmark=tickerToTrade)

PORTFOLIO HASH: 5522786eb28943c781c8bf62503848a2bdf51ba50e612fd13c5a3f9f
bafabbb2ddbb6e294ed821e1e1e41116981d49addb415b95798bb67e
e1111d514a667ab22d8f48a2734a7dab0be1350f8ce70ca3b8a172ac


In [27]:
def getPortfolios():
    while True:
        try:
            datastore_client = datastore.Client('money-maker-1236')
            query = datastore_client.query(kind=params.portfolioLookup)
            retrievedPortfolios = [{
                "key":item.key.name,
                "description":item["description"],
                "benchmark":item["benchmark"]
            } for item in list(query.fetch())]

            return retrievedPortfolios
        except:
            time.sleep(10)
            print("DATA SOURCE RETRIEVAL ERROR:", str(sys.exc_info()))

In [28]:
portfolios = getPortfolios()
print(portfolios)

[{'key': '5522786eb28943c781c8bf62503848a2bdf51ba50e612fd13c5a3f9f', 'description': 'Just trade underlying', 'benchmark': 'EWZ'}, {'key': '7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f', 'description': 'VO 8-13-17', 'benchmark': 'VO'}]


In [29]:
def getPortfolioModels(portfolioKey):
    while True:
        try:
            datastore_client = datastore.Client('money-maker-1236')
            query = datastore_client.query(kind=params.portfolioDB)
            query.add_filter('portfolio', '=', portfolioKey)
            
            retrievedModels = [item["model"] for item in list(query.fetch())]

            return retrievedModels
        except:
            time.sleep(10)
            print("DATA SOURCE RETRIEVAL ERROR:", str(sys.exc_info()))

In [30]:
for item in portfolios:
    print(item)
    print(getPortfolioModels(item["key"]))

{'key': '5522786eb28943c781c8bf62503848a2bdf51ba50e612fd13c5a3f9f', 'description': 'Just trade underlying', 'benchmark': 'EWZ'}
['e1111d514a667ab22d8f48a2734a7dab0be1350f8ce70ca3b8a172ac', 'bafabbb2ddbb6e294ed821e1e1e41116981d49addb415b95798bb67e']
{'key': '7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f', 'description': 'VO 8-13-17', 'benchmark': 'VO'}
['2c69141675447782e2a74036a7cbd8fb5fc08476cebf557bd6e787a6', '0b84734d97e005d3a2a3a06d82cb793ec1f7557573e3bc032fb7fdf3', '11912f289e86b30f1af1b7272e773732dd099e3069cdf79f1c2ba47e', '78afec28d6c4e55102b09d8b6b179a3508e2b03fb46d2e8858c14fdc', '341fe3e8eb88c9fbdd779c6c8727762f99fc1043383dd8242da044e9']


In [31]:
def getModelsByKey(modelHashes):
    while True:
        try:
            datastore_client = datastore.Client('money-maker-1236')
            ##form keys
            keys = []
            for hashing in modelHashes:
                key = datastore_client.key(params.datastoreName, hashing)
                keys.append(key)
                
            retrievedModels = datastore_client.get_multi(keys)
            toReturn = []
            for source in retrievedModels:
                toReturn.append(pickle.loads(source["model"]))
            return toReturn
            
        except:
            time.sleep(10)
            print("DATA SOURCE RETRIEVAL ERROR:", str(sys.exc_info()))

In [32]:
getModelsByKey(getPortfolioModels(item["key"]))